In [1111]:
%%time
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import time
from IPython.display import clear_output
import matplotlib.pyplot as plt
import seaborn as sns
import re

Wall time: 0 ns


In [1112]:
# data = pd.read_csv(r'C:\Users\utilisateur\OneDrive\Documents\Mlearning\Brief_emploi\indeed.csv')
# data = pd.read_csv(r'C:\Users\ThinkCentre\OneDrive\Documents\Mlearning\Brief_emploi\indeed.csv')
data = pd.read_csv(r'C:\Users\utilisateur\OneDrive\Documents\Mlearning\Brief_emploi\new_data.csv')
# data = pd.read_csv(r'C:\Users\ThinkCentre\OneDrive\Documents\Mlearning\Brief_emploi\data.csv')

In [1113]:
data.rename(columns={'lien': 'Url', 'poste': 'Poste', 'entreprise':'Company', 'contrat':'Contrat','lieu':'Lieu','salaire':'Salaire','publication':'Date','description':'Description'}, inplace=True)

In [1114]:
df = data.copy()

## Création de deux dataset avec et sans salaires

In [1115]:
df_test = df.loc[df['Salaire'] == 'false']
df = df.loc[df['Salaire'] != 'false']
df.replace(value=np.NaN, to_replace='false', inplace=True, regex=True)

In [1116]:
(df.isna().sum()/df.shape[0]).sort_values(ascending=True)

_id                  0.000000
Url                  0.000000
Poste                0.000000
Company              0.000000
Lieu                 0.000000
Salaire              0.000000
Date                 0.000000
Description          0.000000
Contrat              0.099111
contrat_recherche    0.354511
metier_recherche     0.354511
ville_recherche      0.354511
dtype: float64

In [1117]:
df_test.isna().sum()

_id                     0
Url                     0
Poste                   0
Company                 0
Contrat                 0
Lieu                    0
Salaire                 0
Date                    0
Description             0
contrat_recherche    2252
metier_recherche     2252
ville_recherche      2252
dtype: int64

In [1118]:
df.isna().sum()

_id                    0
Url                    0
Poste                  0
Company                0
Contrat              156
Lieu                   0
Salaire                0
Date                   0
Description            0
contrat_recherche    558
metier_recherche     558
ville_recherche      558
dtype: int64

## Nettoyage des différentes colonnes du dataset train

## Contrats

In [1119]:
df['Contrat'][df.Contrat.str.contains('CDI', regex= True, na=True)].value_counts()

Temps plein, CDI                                615
CDI                                             480
Temps plein, Temps partiel, CDI                   5
Temps plein, Freelance / Indépendant, CDI         2
Stage, CDI                                        1
Temps plein, CDD, CDI                             1
CDD, CDI, Apprentissage, Contrat pro              1
Temps plein, CDI, Apprentissage, Contrat pro      1
Temps partiel, CDI                                1
Intérim, CDD, CDI                                 1
Freelance / Indépendant, CDI                      1
Name: Contrat, dtype: int64

In [1120]:
df['Contrat'].replace(value='Contrat pro', to_replace='.*(Contrat pro)(.)*', inplace=True, regex=True)

In [1121]:
df['Contrat'].replace(value='Stage', to_replace='.*(Stage)(.)*', inplace=True, regex=True)

In [1122]:
df['Contrat'].replace(value='Freelance / Indépendant', to_replace='.*(Freelance / Indépendant)(.)*', inplace=True, regex=True)

In [1123]:
df['Contrat'].replace(value='Intérim', to_replace='.*(Intérim)(.)*', inplace=True, regex=True)

In [1124]:
df['Contrat'].replace(value='CDD', to_replace='.*(CDD)(.)*', inplace=True, regex=True)

In [1125]:
df['Contrat'].replace(value='CDI', to_replace='.*(CDI)(.)*', inplace=True, regex=True)

In [1126]:
df['Contrat'].replace(value='Apprentissage', to_replace='.*(Apprentissage)(.)*', inplace=True, regex=True)

In [1127]:
df['Contrat'].replace(value='Temps partiel', to_replace='.*(Temps partiel)(.)*', inplace=True, regex=True)

In [1128]:
# df['Contrat'][df.Contrat.str.contains('CDD', regex= True, na=False)].value_counts()

In [1129]:
df['Contrat'].replace(value='Autres', to_replace=np.NaN, inplace=True, regex=True)

In [1130]:
df['contrat_recherche'].replace(value='Autres', to_replace=np.NaN, inplace=True, regex=True)

In [1131]:
df.Contrat.value_counts()

CDI                        1101
Freelance / Indépendant     158
Autres                      156
Stage                        80
CDD                          30
Intérim                      27
Contrat pro                  11
Apprentissage                 8
Temps partiel                 3
Name: Contrat, dtype: int64

## Postes

In [1132]:
liste_postes = data['Poste']
for k in range(len(liste_postes)) :
    if ("develop" in liste_postes[k].lower() or "dévelop" in liste_postes[k].lower() or "devop" in liste_postes[k].lower()
    or "python" in liste_postes[k].lower() or "java" in liste_postes[k].lower() or "tech" in liste_postes[k].lower()
    or "master" in liste_postes[k].lower() or "web" in liste_postes[k].lower()) :

        liste_postes[k] = "Developpeur"

    elif ("analy" in liste_postes[k].lower() or "sql" in liste_postes[k].lower() or "big" in liste_postes[k].lower()) :

        liste_postes[k] = "Data Analyst"

    elif ("scien" in liste_postes[k].lower() or "engineer" in liste_postes[k].lower() or "ingé" in liste_postes[k].lower()
    or "inge" in liste_postes[k].lower() or "archi" in liste_postes[k].lower()) :

        liste_postes[k] = "Data Scientist"

    elif ("business" in liste_postes[k].lower() or "intell" in liste_postes[k].lower() or "market" in liste_postes[k].lower()
    or "consult" in liste_postes[k].lower() or "product" in liste_postes[k].lower() or "manag" in liste_postes[k].lower()
    or "comm" in liste_postes[k].lower()) :

        liste_postes[k] = "Business Intelligence"
    
    else :

        liste_postes[k] = "Autre"


In [1133]:
df['Poste'] = liste_postes

## Metier_recherche

In [1134]:
df['metier_recherche'].replace(value='Autres', to_replace=np.NaN, inplace=True, regex=True)

## Salaires

### Création de 3 colonnes : Salaire min, max , mean

In [1135]:
df['Salaire_min'] = df['Salaire']
df['Salaire_max'] = df['Salaire']
df['Salaire_mean'] = 0

### Nettoyage des 3 colonnes

In [1136]:
df['Salaire_min'].replace(value='', to_replace=r'(par|mois|moi|€|an|jour|heure|semaine|\s)', inplace=True, regex=True)
df['Salaire_max'].replace(value='', to_replace=r'(par|mois|moi|€|an|jour|heure|semaine|\s)', inplace=True, regex=True)

In [1137]:
df['Salaire_min'].replace(value='', to_replace=r'(\-[0-9]*$)', inplace=True, regex=True)

In [1138]:
df['Salaire_max'].replace(value='', to_replace=r'(^[0-9]*[\-])', inplace=True, regex=True)

In [1139]:
liste_min = df['Salaire_min'].value_counts()

In [1140]:
df.drop(df[df.Salaire_min == '{0}{2}'].index, inplace=True)
# df.drop(df[df.Salaire_min == '800-1000semaine'].index, inplace=True)
df.drop(df[df.Salaire_min == '24,72'].index, inplace=True)
df.drop(df[df.Salaire_min == '13,18'].index, inplace=True)
df.drop(df[df.Salaire_min == '20,88'].index, inplace=True)

### Modification pour ramener les salaires par an

In [1141]:
for i,j in enumerate(df['Salaire']):
    j = str(j)
    
    if re.search(r'mois', j):
        a=df['Salaire_min'].iloc[i] 
        a = int(a)
        a=(a*12)        
        df['Salaire_min'].iloc[i] = a
        
        

In [1142]:
for i,j in enumerate(df['Salaire']):
    j = str(j)
    
    if re.search(r'jour', j):
        a=df['Salaire_min'].iloc[i] 
        a = int(a)
        a=(a*228)        
        df['Salaire_min'].iloc[i] = a

In [1143]:
for i,j in enumerate(df['Salaire']):
    j = str(j)
    
    if re.search(r'heure', j):
        a=df['Salaire_min'].iloc[i] 
        a = int(a)
        a=(a*1600)        
        df['Salaire_min'].iloc[i] = a

In [1144]:
for i,j in enumerate(df['Salaire']):
    j = str(j)
    
    if re.search(r'semaine', j):
        a=df['Salaire_min'].iloc[i] 
        a = int(a)
        a=(a*46)        
        df['Salaire_min'].iloc[i] = a

In [1145]:
for i,j in enumerate(df['Salaire']):
    j = str(j)
    
    if re.search(r'mois', j):
        a=df['Salaire_max'].iloc[i] 
        a = int(a)
        a=(a*12)        
        df['Salaire_max'].iloc[i] = a

In [1146]:
for i,j in enumerate(df['Salaire']):
    j = str(j)
    
    if re.search(r'jour', j):
        a=df['Salaire_max'].iloc[i] 
        a = int(a)
        a=(a*228)        
        df['Salaire_max'].iloc[i] = a

In [1147]:
for i,j in enumerate(df['Salaire']):
    j = str(j)
    
    if re.search(r'heure', j):
        a=df['Salaire_max'].iloc[i] 
        a = int(a)
        a=(a*1600)        
        df['Salaire_max'].iloc[i] = a

In [1148]:
for i,j in enumerate(df['Salaire']):
    j = str(j)
    
    if re.search(r'semaine', j):
        a=df['Salaire_max'].iloc[i] 
        a = int(a)
        a=(a*46)        
        df['Salaire_max'].iloc[i] = a

### Remplissage de la colonne mean

In [1149]:
for i,j in enumerate(zip(df['Salaire_min'], df['Salaire_max'])):

    a = (int(j[0]) + int(j[1]))/2
    df['Salaire_mean'].iloc[i] = a

## Date

In [1150]:
df['Date'].replace(value='1', to_replace=".*(Aujourd'hui)(.)*", inplace=True, regex=True)

In [1151]:
df['Date'].replace(value='', to_replace="[a-zA-Z éà']", inplace=True, regex=True)

In [1152]:
df['Date'].replace(value='40', to_replace="(30\+)", inplace=True, regex=True)

In [1153]:
df.replace(value=np.NaN, to_replace='', inplace=True, regex=True)

In [1154]:
df['Date'].dropna(inplace=True)

## Lieu

In [1155]:
df.Lieu.value_counts()

Paris (75)                    599
Lyon (69)                     142
Nantes (44)                   139
Toulouse (31)                 111
Boulogne-Billancourt (92)      32
                             ... 
Pierre-Bénite (69)              1
Saint-Germain-en-Laye (78)      1
Paris 6e (75)                   1
Villejuif (94)                  1
Saint-Fons (69)                 1
Name: Lieu, Length: 133, dtype: int64

In [1156]:
for i,j in enumerate(df['Lieu']):
    j = str(j)
    
    if re.search(r'75|92|94|78|93|77|91|95|Île-de-France', j):
        # a=df['ville_'].iloc[i]              
        df['ville_recherche'].iloc[i] = 'Paris'

In [1157]:
for i,j in enumerate(df['Lieu']):
    j = str(j)
    
    if re.search(r'54', j):
        # a=df['ville_'].iloc[i]              
        df['ville_recherche'].iloc[i] = 'Nancy'

In [1158]:
for i,j in enumerate(df['Lieu']):
    j = str(j)
    
    if re.search(r'69', j):
        # a=df['ville_'].iloc[i]              
        df['ville_recherche'].iloc[i] = 'Lyon'

In [1159]:
for i,j in enumerate(df['Lieu']):
    j = str(j)
    
    if re.search(r'31', j):
        # a=df['ville_'].iloc[i]              
        df['ville_recherche'].iloc[i] = 'Toulouse'

In [1160]:
for i,j in enumerate(df['Lieu']):
    j = str(j)
    
    if re.search(r'44', j):
        # a=df['ville_'].iloc[i]              
        df['ville_recherche'].iloc[i] = 'Nantes'

In [1161]:
df['Lieu'].replace(value='Paris', to_replace='.*(Paris)(.)*', inplace=True, regex=True)
df['Lieu'].replace(value='Lyon', to_replace='.*(Lyon)(.)*', inplace=True, regex=True)
df['Lieu'].replace(value='Bordeau', to_replace='.*(Bordeau)(.)*', inplace=True, regex=True)
df['Lieu'].replace(value='Nancy', to_replace='.*(Nancy)(.)*', inplace=True, regex=True)
df['Lieu'].replace(value='Nantes', to_replace='.*(Nantes)(.)*', inplace=True, regex=True)
df['Lieu'].replace(value='Strasbourg', to_replace='.*(Strasbourg)(.)*', inplace=True, regex=True)
df['Lieu'].replace(value='Toulouse', to_replace='.*(Toulouse)(.)*', inplace=True, regex=True)

In [1162]:
df['Lieu'].replace(value='', to_replace='[0-9()]', inplace=True, regex=True)

In [1163]:
df.Lieu.value_counts()

Paris                    761
Lyon                     205
Nantes                   139
Toulouse                 111
Boulogne-Billancourt      32
                        ... 
Saint-Genis-Laval          1
Malakoff                   1
Nogent-sur-Marne           1
Palaiseau                  1
Charenton-le-Pont          1
Name: Lieu, Length: 105, dtype: int64

## Transformation de ville recherche en Region

In [1164]:
df['ville_recherche'].replace(value='Ile de france', to_replace='Paris', inplace=True, regex=True)
df['ville_recherche'].replace(value='Rhône Alpes', to_replace='Lyon', inplace=True, regex=True)
df['ville_recherche'].replace(value='Aquitaine', to_replace='Bordeau', inplace=True, regex=True)
df['ville_recherche'].replace(value='Lorraine', to_replace='Nancy', inplace=True, regex=True)
df['ville_recherche'].replace(value='Pays de Loire', to_replace='Nantes', inplace=True, regex=True)
df['ville_recherche'].replace(value='Alsace', to_replace='Strasbourg', inplace=True, regex=True)
df['ville_recherche'].replace(value='Midi Pyrénées', to_replace='Toulouse', inplace=True, regex=True)

In [1165]:
df.rename(columns={'ville_recherche': 'Region'}, inplace=True)

## Création d'une colonne code postal

In [1166]:
df['Code_postal'] = df['Region']

In [1167]:
df['Code_postal'].replace(value='75000', to_replace='Ile de france', inplace=True, regex=True)
df['Code_postal'].replace(value='69000', to_replace='Rhône Alpes', inplace=True, regex=True)
df['Code_postal'].replace(value='33000', to_replace='Aquitaine', inplace=True, regex=True)
df['Code_postal'].replace(value='54000', to_replace='Lorraine', inplace=True, regex=True)
df['Code_postal'].replace(value='44000', to_replace='Pays de Loire', inplace=True, regex=True)
df['Code_postal'].replace(value='67000', to_replace='Alsace', inplace=True, regex=True)
df['Code_postal'].replace(value='31000', to_replace='Midi Pyrénées', inplace=True, regex=True)

## Company

In [1168]:
df['Company'].replace(value='', to_replace='\n', inplace=True, regex=True)

## Nettoyage des Na restants

In [1169]:
df.isna().sum()

_id                   0
Url                   0
Poste                 0
Company               0
Contrat               0
Lieu                  0
Salaire               0
Date                 14
Description           0
contrat_recherche     0
metier_recherche      0
Region                0
Salaire_min           0
Salaire_max           0
Salaire_mean          0
Code_postal           0
dtype: int64

In [1170]:
df.dropna(inplace=True)

## Sauvegarde du dataset nettoyé

In [1171]:
df.to_csv(r'C:\Users\utilisateur\OneDrive\Documents\Mlearning\Brief_emploi\Emploi.csv', index=False)

## Création d'une fonction de nettoyage

In [1172]:
def nettoyage_test (df):
    
    df.replace(value=np.NaN, to_replace='false', inplace=True, regex=True)

        
    df['Contrat'].replace(value='Contrat pro', to_replace='.*(Contrat pro)(.)*', inplace=True, regex=True)
    df['Contrat'].replace(value='Stage', to_replace='.*(Stage)(.)*', inplace=True, regex=True)
    df['Contrat'].replace(value='Freelance / Indépendant', to_replace='.*(Freelance / Indépendant)(.)*', inplace=True, regex=True)
    df['Contrat'].replace(value='Intérim', to_replace='.*(Intérim)(.)*', inplace=True, regex=True)
    df['Contrat'].replace(value='CDD', to_replace='.*(CDD)(.)*', inplace=True, regex=True)
    df['Contrat'].replace(value='CDI', to_replace='.*(CDI)(.)*', inplace=True, regex=True)
    df['Contrat'].replace(value='Apprentissage', to_replace='.*(Apprentissage)(.)*', inplace=True, regex=True)   
    df['Contrat'].replace(value='Temps partiel', to_replace='.*(Temps partiel)(.)*', inplace=True, regex=True)     
    df['Contrat'].replace(value='Autres', to_replace=np.NaN, inplace=True, regex=True)
    df['contrat_recherche'].replace(value='Autres', to_replace=np.NaN, inplace=True, regex=True)

    liste_postes = data['Poste']
    for k in range(len(liste_postes)) :
        if ("develop" in liste_postes[k].lower() or "dévelop" in liste_postes[k].lower() or "devop" in liste_postes[k].lower()
        or "python" in liste_postes[k].lower() or "java" in liste_postes[k].lower() or "tech" in liste_postes[k].lower()
        or "master" in liste_postes[k].lower() or "web" in liste_postes[k].lower()) :

            liste_postes[k] = "Developpeur"

        elif ("analy" in liste_postes[k].lower() or "sql" in liste_postes[k].lower() or "big" in liste_postes[k].lower()) :

            liste_postes[k] = "Data Analyst"

        elif ("scien" in liste_postes[k].lower() or "engineer" in liste_postes[k].lower() or "ingé" in liste_postes[k].lower()
        or "inge" in liste_postes[k].lower() or "archi" in liste_postes[k].lower()) :

            liste_postes[k] = "Data Scientist"

        elif ("business" in liste_postes[k].lower() or "intell" in liste_postes[k].lower() or "market" in liste_postes[k].lower()
        or "consult" in liste_postes[k].lower() or "product" in liste_postes[k].lower() or "manag" in liste_postes[k].lower()
        or "comm" in liste_postes[k].lower()) :

            liste_postes[k] = "Business Intelligence"
        
        else :

            liste_postes[k] = "Autre"

    df['Poste'] = liste_postes

    df['metier_recherche'].replace(value='Autres', to_replace=np.NaN, inplace=True, regex=True)

    df['Date'].replace(value='1', to_replace=".*(Aujourd'hui)(.)*", inplace=True, regex=True)
    df['Date'].replace(value='', to_replace="[a-zA-Z éà']", inplace=True, regex=True)
    df['Date'].replace(value='40', to_replace="(30\+)", inplace=True, regex=True)

    df.replace(value=np.NaN, to_replace='', inplace=True, regex=True)
    # df['Date'].dropna(inplace=True)
    # df['Contrat'].dropna(inplace=True)

    for i,j in enumerate(df['Lieu']):
        j = str(j)
    
        if re.search(r'75|92|94|78|93|77|91|95|Île-de-France', j):
            # a=df['ville_'].iloc[i]              
            df['ville_recherche'].iloc[i] = 'Paris'
    
    for i,j in enumerate(df['Lieu']):
        j = str(j)
    
        if re.search(r'54', j):
            # a=df['ville_'].iloc[i]              
            df['ville_recherche'].iloc[i] = 'Nancy'
    
    for i,j in enumerate(df['Lieu']):
        j = str(j)
    
        if re.search(r'69', j):
            # a=df['ville_'].iloc[i]              
            df['ville_recherche'].iloc[i] = 'Lyon'
    
    for i,j in enumerate(df['Lieu']):
        j = str(j)
    
        if re.search(r'31', j):
            # a=df['ville_'].iloc[i]              
            df['ville_recherche'].iloc[i] = 'Toulouse'
    
    for i,j in enumerate(df['Lieu']):
        j = str(j)
    
        if re.search(r'44', j):
            # a=df['ville_'].iloc[i]              
            df['ville_recherche'].iloc[i] = 'Nantes'
    
    df['Lieu'].replace(value='Paris', to_replace='.*(Paris)(.)*', inplace=True, regex=True)
    df['Lieu'].replace(value='Lyon', to_replace='.*(Lyon)(.)*', inplace=True, regex=True)
    df['Lieu'].replace(value='Bordeau', to_replace='.*(Bordeau)(.)*', inplace=True, regex=True)
    df['Lieu'].replace(value='Nancy', to_replace='.*(Nancy)(.)*', inplace=True, regex=True)
    df['Lieu'].replace(value='Nantes', to_replace='.*(Nantes)(.)*', inplace=True, regex=True)
    df['Lieu'].replace(value='Strasbourg', to_replace='.*(Strasbourg)(.)*', inplace=True, regex=True)
    df['Lieu'].replace(value='Toulouse', to_replace='.*(Toulouse)(.)*', inplace=True, regex=True)
    df['Lieu'].replace(value='', to_replace='[0-9()]', inplace=True, regex=True)

    df['ville_recherche'].replace(value='Ile de france', to_replace='Paris', inplace=True, regex=True)
    df['ville_recherche'].replace(value='Rhône Alpes', to_replace='Lyon', inplace=True, regex=True)
    df['ville_recherche'].replace(value='Aquitaine', to_replace='Bordeau', inplace=True, regex=True)
    df['ville_recherche'].replace(value='Lorraine', to_replace='Nancy', inplace=True, regex=True)
    df['ville_recherche'].replace(value='Pays de Loire', to_replace='Nantes', inplace=True, regex=True)
    df['ville_recherche'].replace(value='Alsace', to_replace='Strasbourg', inplace=True, regex=True)
    df['ville_recherche'].replace(value='Midi Pyrénées', to_replace='Toulouse', inplace=True, regex=True)

    df.rename(columns={'ville_recherche': 'Region'}, inplace=True)

    df['Code_postal'] = df['Region']

    df['Code_postal'].replace(value='75000', to_replace='Ile de france', inplace=True, regex=True)
    df['Code_postal'].replace(value='69000', to_replace='Rhône Alpes', inplace=True, regex=True)
    df['Code_postal'].replace(value='33000', to_replace='Aquitaine', inplace=True, regex=True)
    df['Code_postal'].replace(value='54000', to_replace='Lorraine', inplace=True, regex=True)
    df['Code_postal'].replace(value='44000', to_replace='Pays de Loire', inplace=True, regex=True)
    df['Code_postal'].replace(value='67000', to_replace='Alsace', inplace=True, regex=True)
    df['Code_postal'].replace(value='31000', to_replace='Midi Pyrénées', inplace=True, regex=True)

    df['Company'].replace(value='', to_replace='\n', inplace=True, regex=True)

    df.drop(columns='Salaire',inplace=True,axis=1)
    df.dropna(inplace=True)

    df.to_csv(r'C:\Users\utilisateur\OneDrive\Documents\Mlearning\Brief_emploi\Emploi_test.csv', index=False)

    return df


## Nettoyage du dataset test

In [1173]:
test = nettoyage_test(df_test)

In [1174]:
test.isna().sum()

_id                  0
Url                  0
Poste                0
Company              0
Contrat              0
Lieu                 0
Date                 0
Description          0
contrat_recherche    0
metier_recherche     0
Region               0
Code_postal          0
dtype: int64